In [ ]:
import os
import random
from pathlib import Path
import pandas as pd
import cv2
import numpy as np

import yaml


In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


In [ ]:
def ConvertLisaToYolo(dataSetFolderPath: str ):
    # annotations = []
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=',')
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    uniqueFiles = annotationsDataFrame['filename'].unique()
    for file in uniqueFiles:
        image_file = os.path.join(dataSetFolderPath, file)
        if not os.path.exists(image_file):
            continue
        fileDataFrame = annotationsDataFrame[annotationsDataFrame['filename'] == file]
        fileAnnotations = []
        for index, row in fileDataFrame.iterrows():

            image = cv2.imread(dataSetFolderPath+"/"+row[0])

            resizedImage, x1, y1, x2, y2 = ResizeImage(image, row[1], row[2], row[3], row[4], 640, 640)
            # x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
            h, w, _ = image.shape
            xCenter = ((x1 + x2) // 2) / w
            yCenter = ((y1 + y2) // 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h
            classId = uniqueSigns.index(row[5])
            cv2.imwrite("./newYoloV5DataSet"+"/"+file.split("/")[-1], resizedImage)
            fileAnnotations.append(f"{classId} {xCenter} {yCenter} {width} {height}")
        # filepath = "./YoloV5DataSet/"+"/".join(file.split("/")[:-1])
        # print(filepath)
        with open(os.path.join('./newYoloV5DataSet', f"{Path(file).stem}.txt"), "w") as f:
            f.write("\n".join(fileAnnotations))

In [ ]:
def CreateTrainValFiles(dataSetFolderPath: str):
    annotationsFilePath = "./DataSet2/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=',')
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    uniqueFiles = annotationsDataFrame['filename'].unique().tolist()
    image_files = [dataSetFolderPath+"/"+f for f in os.listdir(dataSetFolderPath) if f.endswith('.jpg') or f.endswith('.png')]

    # image_files = ["./DataSet/" + file for file in uniqueFiles ]
    random.shuffle(image_files)

    split_ratio = 0.8
    train_files = image_files[:int(len(image_files) * split_ratio)]
    val_files = image_files[int(len(image_files) * split_ratio):]

    with open("train.txt", "w") as f:
        f.writelines([ file + "\n" for file in train_files])

    with open("val.txt", "w") as f:
        f.writelines([file + "\n" for file in val_files])
    return len(uniqueSigns), uniqueSigns


In [ ]:

def CreateYaml(fileName: str, numClass: int, classList: list):
    data = {
    'train': './train.txt',
    'val': './val.txt',
    'nc': numClass,
    'names': classList
    }

    with open(fileName, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [ ]:
os.mkdir('./newYoloV5DataSet')

In [ ]:
ConvertLisaToYolo('./DataSet2')
numClass, classList = CreateTrainValFiles('./newYoloV5DataSet')
CreateYaml('signboard.yaml', numClass, classList)